#### Gridsearch has been postponed in this notebook since it was taking up a lot of RAM space for the large dataset we have.

In [1]:
import sys
loader_util_path = r"/home/mhasan3/Desktop/WorkFolder/"
sys.path.append(loader_util_path)

In [2]:
import h5py
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [3]:
# import the necessary keras packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from loader_util.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from imutils import paths

In [4]:
# import scoring and utility functions from sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, ShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs
#
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [5]:
# construct the argument parser
data_dir = r"/home/mhasan3/Desktop/WorkFolder/cats_vs_dogs/"

args = {
    "dataset": f"{data_dir}//hdf5//extracted_features.hdf5",
    "model": f"{data_dir}//outputs//extracted_features_lr_model.pt",
    "jobs": -1
}

In [6]:
# open the hdf5 dataset to get the extracted features
db = h5py.File(args["dataset"], "r")
i = int(db["labels"].shape[0] * 0.75)

In [7]:
# define the set of gridsearch parameters
print(f'[INFO] tuning hyperparameters.....')
# params = {"C": [0.0001, 0.001, 0.01, 0.1, 1.0]}
# model = GridSearchCV(LogisticRegression(solver="lbfgs",
#                                         multi_class="auto"),
#                      param_grid=params,
#                      cv=3,
#                      n_jobs=args["jobs"])
# model.fit(db["extracted_features"][:i], db["labels"][:i])
# print(f'[INFO] best hyperparameters are: {model.best_params_}.....')

lr = LogisticRegression()
fit = lr.fit(db["extracted_features"][:i], db["labels"][:i])

[INFO] tuning hyperparameters.....


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
target_names = [cat.decode() for cat in db["label_names"][:]]
target_names

['cat', 'dog']

In [13]:
print(f'[INFO] evaluating.....')
model = lr
preds = model.predict(db["extracted_features"][i:])
print(classification_report(db["labels"][i:], preds,
                            target_names=target_names))

# compute the raw accuracy with extra precision
acc = accuracy_score(db["labels"][i:], preds)
print(f'[INFO] score: {acc}.....')

[INFO] evaluating.....
              precision    recall  f1-score   support

         cat       0.99      0.99      0.99      3118
         dog       0.99      0.99      0.99      3132

    accuracy                           0.99      6250
   macro avg       0.99      0.99      0.99      6250
weighted avg       0.99      0.99      0.99      6250

[INFO] score: 0.98912.....


In [15]:
# serialise the model to disk
print(f'[INFO] saving model.....')
f = open(args["model"], "wb")
f.write(pickle.dumps(model))
f.close()
db.close()

[INFO] saving model.....
